In [1]:
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
import numpy as np
import pandas as pd
import pylab as plt

# Project

### Description

This is a learning project. The point of it is to run analysis on the HoI4 air combat algorithm. It will use python and MatPlotLib to make the calculations and display them as graphs to make analysis and comparison easier.

**Learning note 1**

Remember to run the code snippets with Shift-Enter. They also needs to be run in the correct order to update the scope when you make 

### Preparation

We need to set up the basic data that we will be running the algorithm over to see the results. We will be using objects to store and pass data. 

In [3]:
class Plane:
    def __init__(self, attack, defence, agility, speed, cost):
        self.attack = attack
        self.defence = defence
        self.agility = agility
        self.speed = speed
        self.cost = cost
        
    @property
    def attack(self):
        return self._attack
    @property
    def defence(self):
        return self._defence
    @property
    def agility(self):
        return self._agility
    @property
    def speed(self):
        return self._speed
    @property
    def cost(self):
        return self._cost
    @attack.setter
    def attack(self, val: int):
        self._attack = val
    @defence.setter
    def defence(self, val: int):
        self._defence = val
    @agility.setter
    def agility(self, val: int):
        self._agility = val
    @speed.setter
    def speed(self, val: int):
        self._speed = val
    @cost.setter
    def cost(self, val: int):
        self._cost = val


This is the primary class we will be using. The properties are the values for the planes that we will be comparing.

**Learning note 2**

This can either be done only with the top block after def `__init__`. The next block is a long winded way to make get/set methods. This enables things like input validation and typing. I have skipped input validation and typing here for the time being.

**Learning Note 3**

The underscore in the setters and getters seem to be important. Without them trying to use the object kept killing the kernel. It turns out that it differentiates between the property and the method. Without the underscore det get method calls the get method and loops.

In [4]:
class VariantPlane():
    def __init__(self, plane: Plane, weapons: int = 0, pRange: int = 0, engine: int = 0, reliability: int = 0):
        self.plane = plane
        self.weapons = weapons # +10% attack, - 1% speed, 10% Reliability, 5% agility
        self.pRange = pRange # +10% Range, -10% Reliability
        self.engine = engine # +2% speed, 5% agility
        self.reliability = reliability # +10% Reliabilty
        
    @property
    def attack(self):
        return self.plane.attack*(1+0.1*self._weapons)
    @property
    def defence(self):
        return self.plane.defence
    @property
    def agility(self):
        return self.plane.agility-(0.05*self._weapons*self.plane.agility)+(0.05*self.plane.agility*self.engine)
    @property
    def speed(self):
        return self.plane.speed-(0.01*self._weapons*self.plane.speed)+(0.02*self.engine*self.plane.speed)
    @property
    def cost(self):
        return self.plane.cost

    @property
    def weapons(self):
        return self._weapons
    @weapons.setter
    def weapons(self, val: int):
        if not isinstance(val, int):
            raise TypeError("weapons should be an int")
        if val > 5:
            val = 5
        self._weapons = abs(val)
    @property
    def pRange(self):
        return self._pRange
    @pRange.setter
    def pRange(self, val: int):
        if not isinstance(val, int):
            raise TypeError("range should be an int")
        if val > 5:
            val = 5
        self._pRange = abs(val)
    @property
    def engine(self):
        return self._engine
    @engine.setter
    def engine(self, val: int):
        if not isinstance(val, int):
            raise TypeError("engine should be an int")
        if val > 5:
            val = 5
        self._engine = abs(val)
    @property
    def reliability(self):
        return self._reliability
    @reliability.setter
    def reliability(self, val: int):
        if not isinstance(val, int):
            raise TypeError("reliability should be an int")
        if val > 5:
            val = 5
        self._reliability = abs(val)

I will come back to figure out this later. The point is that I want to make variants to planes without having to type in everything. I will send in the plane I want a variant of and then overwrite the specific values.

I see that this might not be the best way to do this with this specific dataset. The upgrades are percentage bonus/penalties. That means that I want the original plane values and the upgrade points, and afterwards apply the bonuses.

I also want to be able to only set one of the possible variant values, and have the others initiate as Zero.

### Later
Asked a question and got some help. Was advised to make the variant with nested objects. That is just have a Plane object in the variable. Using getters and setters here so that the planes values kan be modified when asked for later.

In [66]:
def damage(attacker, defender):
    # damage = 0.01*number of attackers*air attack/air defence*(1+stats multiplier)*carrier factor*(1+agility disadvantage)
    # multiplier = (((attacker.speed-defender.speed)/1500+(attacker.agility-defender.agility)/100)*0.3)
    # print(multiplier)
    
    def statsMultiplier(asp, ds, aa, da):
        multiplier = (((asp-ds)/1500)+(aa-da)/100)
        if multiplier < 1:
            if multiplier < -1:
                return -1*0.3
            return multiplier*0.3
        else: 
            return 1*0.3
    
    def agilityDisadvantage(aa, da):
        disadvantage = 0.45*((aa/da) - 1)
        
        if disadvantage < 0:
            if disadvantage < -0.675:
                return -0.675
            return disadvantage
        else: 
            return 0
    
    disadvantage = agilityDisadvantage(attacker.agility, defender.agility)
    print("disadvantage " + str(disadvantage))
    multiplier = statsMultiplier(attacker.speed, defender.speed, attacker.agility, defender.agility)
    print("multiplier " + str(multiplier))
    return 0.01*1*(attacker.attack/defender.defence)*(1+multiplier)*0.1*(1+disadvantage)

I have been doing this iteratively. First step was doing the stats multiplier calculation. I see that there might be an issue using variantPlane in this. I need to find out if I can have a get method to return variantPlane.attack. Looking over the code that was suggested for variantPlane I can. And it works.

**Learning note 4**

Check up on underscore in Python and what it means.

In [67]:
def test1():
    return (((500-430)/1500+(50-20)/100)*0.3)

print("Test 1 " +str(test1()))

def test2():
    return (((500-650)/1500+(50-65)/100)*0.3)

def test3():
    return (((650-500)/1500+(65-50)/100)*0.3)

def test4():
    return ((((650-500)/1500)+(65-50)/100)*0.3)

print("Test 2 " +str(test2()))

aa = 20
da = 50
print(1+(0.45*(aa/da - 1)))

print(1+(0.45*(da/aa - 1)))

print(((500-430)/1500+(50-20)/100)+0.3)

print("test3 " + str(test3()))
print("test4 " + str(test4()))

def statsMultiplier(asp, ds, aa, da):
    multiplier = (((asp-ds)/1500)+(aa-da)/100)
    if multiplier < 1:
        if multiplier < -1:
            return -1*0.3
        return multiplier*0.3
    else: 
        return 1*0.3


print("Stats multiplier " + str(statsMultiplier(650, 500, 65, 50)))
print("Stats multiplier " + str(statsMultiplier(500, 650, 50, 65)))

Test 1 0.104
Test 2 -0.075
0.73
1.675
0.6466666666666667
test3 0.075
test4 0.075
Stats multiplier 0.075
Stats multiplier -0.075


In [68]:
f33 = Plane(9,8,20,430,22)
f36 = Plane(18,10,50,500,24)
f40 = Plane(27,12,65,650,26)
h40 = Plane(46, 15, 30,550,30)

print("Damage f40vsf36 " + str(damage(f40, f36)) + "\n")
print("Damage f36vsf40 " + str(damage(f36, f40)) + "\n")

print("Damage h40vsf40 " + str(damage(h40, f40)) + "\n")
print("Damage f40vsh40 " + str(damage(f40, h40)) + "\n")



disadvantage 0
multiplier 0.075
Damage f40vsf36 0.0029025000000000006

disadvantage -0.10384615384615382
multiplier -0.075
Damage f36vsf40 0.0012434134615384616

disadvantage -0.2423076923076923
multiplier -0.12499999999999999
Damage h40vsf40 0.0025414262820512823

disadvantage 0
multiplier 0.12499999999999999
Damage f40vsh40 0.0020250000000000003



In [27]:
f33v1 = VariantPlane(plane=f33, weapons=50, pRange=-2, engine=2, reliability=3)

print(f33.attack)
print(f33v1.attack)
print(f33.agility)
print(f33v1.agility)
print(f33.speed)
print(f33v1.speed)

9
13.5
20
17.0
430
425.7


In [20]:
f33v1.__dict__

{'plane': <__main__.Plane at 0x19fc8edadc0>,
 '_weapons': 2,
 '_pRange': 2,
 '_reliability': 3}

In [21]:
f33v1.plane.__dict__

{'_attack': 9, '_defence': 8, '_agility': 45, '_speed': 330, '_cost': 22}